# Electrical Symbol Detection - Inference
Run predictions on floor plans using the trained FasterRCNN model

## 0. Environment Setup

In [26]:
import sys
import os

# Mount Google Drive (for checkpoints and dataset)
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=False)
        print("✓ Google Drive mounted at /content/drive")
        DRIVE_MOUNTED = True
    except Exception as e:
        print(f"⚠ Could not mount Google Drive: {e}")
        DRIVE_MOUNTED = False
else:
    print("⚠ Running locally (not on Google Colab)")
    DRIVE_MOUNTED = False

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✓ Google Drive mounted at /content/drive


In [27]:
import subprocess
import os

# Clone or update repository
repo_path = '/content/symbol-detection'

if not os.path.exists(repo_path):
    print("Cloning repository...")
    subprocess.run(['git', 'clone', 'https://github.com/BhanukaDev/symbol-detection.git', repo_path], check=True)
    print(f"✓ Repository cloned to {repo_path}")
else:
    print("Repository already exists, pulling latest changes...")
    os.chdir(repo_path)
    subprocess.run(['git', 'pull'], check=True)
    print("✓ Repository updated")

os.chdir('/content/symbol-detection/python')
print(f"✓ Working directory: {os.getcwd()}")

Repository already exists, pulling latest changes...
✓ Repository updated
✓ Working directory: /content/symbol-detection/python


In [28]:
import importlib
import os
import sys

# First ensure dependencies are installed
print("Installing dependencies...")
!pip install -q torch torchvision pycocotools timm torchmetrics

# Install/update the package (skip dependencies since they're installed)
print("\nInstalling symbol-detection package...")
!pip install --no-deps --force-reinstall -q -e /content/symbol-detection/python/

# Force reload everything
print("\nReloading modules...")
if 'symbol_detection' in sys.modules:
    del sys.modules['symbol_detection']
if 'symbol_detection.inference' in sys.modules:
    del sys.modules['symbol_detection.inference']
if 'symbol_detection.inference.predictor' in sys.modules:
    del sys.modules['symbol_detection.inference.predictor']

# Import fresh
from symbol_detection.inference.predictor import SymbolDetectionPredictor

print("✓ Package installed and reloaded")
print("✓ SymbolDetectionPredictor ready")

Installing dependencies...

Installing symbol-detection package...
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for symbol-detection (pyproject.toml) ... done

Reloading modules...
✓ Package installed and reloaded
✓ SymbolDetectionPredictor ready


## 1. Setup and Imports

In [29]:
import sys
from pathlib import Path

# Setup paths
sys.path.insert(0, '/content/symbol-detection/python/src')

# Define checkpoint and dataset directories
if 'google.colab' in sys.modules:
    # Colab environment
    checkpoint_dir = '/content/drive/MyDrive/symbol-detection/checkpoints'
    dataset_dir = '/content/drive/MyDrive/symbol-detection/dataset'
    os.chdir('/content/symbol-detection/python')
else:
    # Local environment
    checkpoint_dir = str(Path.cwd().parent.parent / 'checkpoints')
    dataset_dir = str(Path.cwd().parent.parent / 'dataset')

# Find the latest or final checkpoint
checkpoint_path = Path(checkpoint_dir) / 'model_epoch_final.pth'

# If final checkpoint doesn't exist, look for latest epoch
if not checkpoint_path.exists():
    import glob
    checkpoints = sorted(glob.glob(str(Path(checkpoint_dir) / 'model_epoch_*.pth')))
    if checkpoints:
        checkpoint_path = Path(checkpoints[-1])
        print(f"Final checkpoint not found, using latest: {checkpoint_path.name}")
    else:
        raise FileNotFoundError(f"No checkpoints found in {checkpoint_dir}")

print(f"✓ Paths configured")
print(f"✓ Checkpoint dir: {checkpoint_dir}")
print(f"✓ Checkpoint file: {checkpoint_path.name}")
print(f"✓ Dataset dir: {dataset_dir}")

✓ Paths configured
✓ Checkpoint dir: /content/drive/MyDrive/symbol-detection/checkpoints
✓ Checkpoint file: model_epoch_final.pth
✓ Dataset dir: /content/drive/MyDrive/symbol-detection/dataset


In [30]:
# Import libraries
import cv2
import json
import numpy as np
import torch
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from symbol_detection.inference import SymbolDetectionPredictor

print("✓ All imports loaded")

✓ All imports loaded


## 2. Load Trained Model

In [ ]:
# Load the model
print("Loading model from checkpoint...")
print(f"Checkpoint path: {checkpoint_path}")
print(f"Checkpoint exists: {checkpoint_path.exists()}")
print(f"Checkpoint size: {checkpoint_path.stat().st_size / 1e6:.1f} MB")

# Create predictor with the loaded SymbolDetectionPredictor
# Note: num_classes=8 includes background class (7 symbols + 1 background)
try:
    predictor = SymbolDetectionPredictor(
        checkpoint_path=str(checkpoint_path),
        num_classes=8,  # 7 symbols + 1 background
        device='cuda' if torch.cuda.is_available() else 'cpu',
        confidence_threshold=0.5
    )
    print("✓ Model loaded successfully!")
    print(f"✓ Classes: {list(predictor.categories.values())}")
except Exception as e:
    print(f"✗ Error loading model: {e}")
    import traceback
    traceback.print_exc()
    raise

Loading model from checkpoint...
Checkpoint path: /content/drive/MyDrive/symbol-detection/checkpoints/model_epoch_final.pth
Checkpoint exists: True
Checkpoint size: 330.3 MB
Loading model from /content/drive/MyDrive/symbol-detection/checkpoints/model_epoch_final.pth...
✓ Model loaded on cuda
✓ Confidence threshold: 0.5
✓ Model loaded successfully!
✗ Error loading model: 'SymbolDetectionPredictor' object has no attribute 'class_names'


Traceback (most recent call last):
  File "/tmp/ipython-input-2765204242.py", line 17, in <cell line: 0>
    print(f"✓ Classes: {predictor.class_names}")
                        ^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'SymbolDetectionPredictor' object has no attribute 'class_names'


AttributeError: 'SymbolDetectionPredictor' object has no attribute 'class_names'

## 3. Inference on Test Images

In [ ]:
# Get test images from dataset
images_dir = Path(dataset_dir) / 'images'
test_images = sorted(list(images_dir.glob('*.png')))[:5]  # Use first 5 images

print(f"Found {len(test_images)} test images")
print(f"Processing {len(test_images)} samples...\n")

# Run inference
results = []
for img_path in test_images:
    image = cv2.imread(str(img_path))
    if image is None:
        continue
    
    # Run prediction
    detections = predictor.predict(image, conf_threshold=0.50)
    
    result = {
        'image': img_path.name,
        'num_detections': len(detections),
        'detections': detections,
        'image_shape': image.shape,
    }
    results.append(result)
    
    print(f"{img_path.name}: {len(detections)} symbols detected")

print(f"\n✓ Inference complete")

## 4. Visualize Detections

In [ ]:
# Visualize detections with matplotlib
fig, axes = plt.subplots(len(test_images), 1, figsize=(12, 4 * len(test_images)))
if len(test_images) == 1:
    axes = [axes]

for idx, (img_path, result) in enumerate(zip(test_images, results)):
    image = cv2.imread(str(img_path))
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Create annotated image
    annotated = predictor.visualize(image, result['detections'])
    annotated_rgb = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
    
    # Display
    axes[idx].imshow(annotated_rgb)
    axes[idx].set_title(f"{img_path.name} - {result['num_detections']} symbols detected")
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

print("✓ Visualization complete")

## 5. Detailed Detection Results

In [ ]:
# Show detailed results for first image
print("Detection Details for First Image:\n")
print(f"Image: {results[0]['image']}")
print(f"Total Detections: {results[0]['num_detections']}\n")

for i, det in enumerate(results[0]['detections'], 1):
    print(f"{i}. {det['class_name']}")
    print(f"   Confidence: {det['confidence']:.3f}")
    print(f"   Box: ({det['bbox'][0]:.0f}, {det['bbox'][1]:.0f}) -> ({det['bbox'][2]:.0f}, {det['bbox'][3]:.0f})")
    print(f"   Size: {det['width']:.0f} x {det['height']:.0f}")
    print()

# Statistics
print("\nInference Statistics:")
total_detections = sum(r['num_detections'] for r in results)
avg_detections = total_detections / len(results) if results else 0
print(f"  Total images: {len(results)}")
print(f"  Total detections: {total_detections}")
print(f"  Average detections per image: {avg_detections:.1f}")

# Detection breakdown by class
class_counts = {}
for result in results:
    for det in result['detections']:
        class_name = det['class_name']
        class_counts[class_name] = class_counts.get(class_name, 0) + 1

print("\n  Detections by Class:")
for class_name, count in sorted(class_counts.items()):
    print(f"    {class_name}: {count}")

## 6. Inference Performance Benchmarking

In [ ]:
import time
import torch

# Benchmark inference speed
print("Benchmarking inference performance...\n")

test_image = cv2.imread(str(test_images[0]))
num_runs = 10

times = []
for i in range(num_runs):
    start = time.time()
    _ = predictor.predict(test_image)
    elapsed = time.time() - start
    times.append(elapsed * 1000)  # Convert to ms

avg_time = np.mean(times)
std_time = np.std(times)
min_time = np.min(times)
max_time = np.max(times)

print(f"Inference Speed ({num_runs} runs):")
print(f"  Average: {avg_time:.2f} ms")
print(f"  Std Dev: {std_time:.2f} ms")
print(f"  Min: {min_time:.2f} ms")
print(f"  Max: {max_time:.2f} ms")
print(f"  Throughput: {1000/avg_time:.1f} images/sec")

# GPU Memory
if torch.cuda.is_available():
    print(f"\nGPU Memory:")
    print(f"  Allocated: {torch.cuda.memory_allocated() / 1024**2:.1f} MB")
    print(f"  Reserved: {torch.cuda.memory_reserved() / 1024**2:.1f} MB")
    
    device_name = torch.cuda.get_device_name(0)
    print(f"  Device: {device_name}")

# Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of inference times
ax1.hist(times, bins=20, edgecolor='black', alpha=0.7)
ax1.set_xlabel('Inference Time (ms)')
ax1.set_ylabel('Frequency')
ax1.set_title(f'Inference Time Distribution ({num_runs} runs)')
ax1.axvline(avg_time, color='red', linestyle='--', linewidth=2, label=f'Mean: {avg_time:.2f}ms')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Performance summary
summary_text = f"""
Inference Performance Summary

Average: {avg_time:.2f} ms
Latency: ±{std_time:.2f} ms
Throughput: {1000/avg_time:.1f} img/s

Model: FasterRCNN ResNet50+FPN
Backbone: Frozen
Device: {predictor.device.upper()}
Confidence: {predictor.confidence_threshold}
"""

ax2.text(0.1, 0.5, summary_text, fontsize=12, family='monospace',
         transform=ax2.transAxes, verticalalignment='center')
ax2.axis('off')

plt.tight_layout()
plt.show()

print("\n✓ Benchmark complete")